In [1]:
import logging
import pathlib
import sys
from typing import List
import random
import os
import time
from subprocess import check_output
import scipy.sparse as sparse

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [78]:
mallet_path = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/Mallet-202108/bin/mallet")

def infer_thetas(path_model, num_topics, docs):
    num_iterations = 1000
    doc_topic_thr = 0.0
    holdout_corpus = path_model / "infer_data" / "corpus.txt"
    with holdout_corpus.open("w", encoding="utf8") as fout:
        for i, (identifier, text) in enumerate(docs):
            clean_text = f"{identifier} {text}".replace('\n', ' ').replace('\r', '')
            fout.write(f"{i} 0 {clean_text}\n")
        print(f"-- -- Mallet corpus.txt for inference created.")

    # Get inferencer
    inferencer = path_model / "model_data" / "inferencer.mallet"

    # Files to be generated thoruogh Mallet
    corpus_mallet_inf = path_model / "infer_data" / "corpus_inf.mallet"
    doc_topics_file = path_model / "infer_data" / "doc-topics-inf.txt"


    # Extract pipe
    # Get corpus file
    path_corpus = path_model / "train_data" / "corpus.mallet"
    if not path_corpus.is_file():
        print(f"-- Pipe extraction: Could not locate corpus file")

    # Create auxiliary file with only first line from the original corpus file
    path_txt = path_model / "train_data" / "corpus.txt"
    with path_txt.open('r', encoding='utf8') as f:
        first_line = f.readline()
    path_aux = path_model / "train_data" / "corpus_aux.txt"
    with path_aux.open('w', encoding='utf8') as fout:
        fout.write(first_line + '\n')

    # We perform the import with the only goal to keep a small file containing the pipe
    print(f"-- Extracting pipeline")
    path_pipe = path_model / "train_data" / "import.pipe"

    cmd = mallet_path.as_posix() + \
        ' import-file --use-pipe-from %s --input %s --output %s'
    cmd = cmd % (path_corpus, path_aux, path_pipe)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- Failed to extract pipeline. Revise command')

    # Import data to mallet
    print('-- Inference: Mallet Data Import')

    #
    cmd = mallet_path.as_posix() + \
        ' import-file --use-pipe-from %s --input %s --output %s'
    cmd = cmd % (path_pipe, holdout_corpus, corpus_mallet_inf)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except Exception as e:
        print(e)
        print('-- Mallet failed to import data. Revise command')

    # Get topic proportions
    print('-- Inference: Inferring Topic Proportions')

    cmd = mallet_path.as_posix() + \
        ' infer-topics --inferencer %s --input %s --output-doc-topics %s ' + \
        ' --doc-topics-threshold ' + str(doc_topic_thr) + \
        ' --num-iterations ' + str(num_iterations)
    cmd = cmd % (inferencer, corpus_mallet_inf, doc_topics_file)

    try:
        print(f'-- Running command {cmd}')
        check_output(args=cmd, shell=True)
    except:
        print('-- Mallet inference failed. Revise command')

    cols = [k for k in np.arange(2, num_topics + 2)]
    thetas32 = np.loadtxt(doc_topics_file, delimiter='\t', dtype=np.float32, usecols=cols)
    thetas32[thetas32 < 3e-3] = 0
    thetas32 = normalize(thetas32, axis=1, norm='l1')
    thetas32 = sparse.csr_matrix(thetas32, copy=True)
    
    path_save = path_model / "infer_data" / "thetas.npz"
    sparse.save_npz(path_save, thetas32)
    
    return thetas32.shape  

In [13]:
################
# Paths to data
################
path_parquets = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed")
path_place_CPV = path_parquets / "completo_CPV.parquet"
path_place_esp = path_parquets / "df_esp_langid.parquet"
path_manual_stops = "/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/stopwords_sin_duplicados"
path_eq = "/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/eq.txt"

In [14]:
################
# Read data
################
#print(f"-- -- Reading data from {path_place_esp} and {path_place_CPV}")
processed = pd.read_parquet(path_place_esp)
#cols = processed.columns.values.tolist()
#print(f"-- -- Data read from {path_place_esp}: {len(processed)} rows.")
#print(f"-- -- Columns: {cols}")
path_place_CPV = pd.read_parquet(path_place_CPV)
#print(f"-- -- Data read from {path_place_CPV}: {len(path_place_CPV)} rows.")
#path_place_CPV.reset_index(inplace=True)

#####################
# Filter stops /eqs #
#####################
# Filter stops
stopwords = set()
# Lista para registrar los nombres de los archivos procesados
archivos_procesados = []
# Iterar sobre cada archivo en el directorio especificado
for archivo in os.listdir(path_manual_stops):
    if archivo.endswith('.txt'):
        ruta_completa = os.path.join(path_manual_stops, archivo)
        with open(ruta_completa, 'r', encoding='utf-8') as f:
            stopwords.update(f.read().splitlines())
        # Registrar el archivo procesado
        archivos_procesados.append(archivo)
print(f"-- -- There are {len(stopwords)} stopwords")
def eliminar_stopwords(fila):
    return ' '.join([palabra for palabra in fila.split() if palabra not in stopwords])
start = time.time()
processed['lemmas'] = processed['lemmas'].apply(eliminar_stopwords)
print(f"-- -- Stops filtered in {time.time() - start}")

# Filter eqs
start = time.time()
pares_diccionario = {}
compiled_regexes = {}
with open(path_eq, 'r') as archivo:
    for linea in archivo:
        linea = linea.strip()
        palabras = linea.split(':')
        if len(palabras) < 2:
            print(f"Línea omitida o incompleta: '{linea}'")
            continue
        pares_diccionario[palabras[0]] = palabras[1]
pares_diccionario = \
    dict(sorted(pares_diccionario.items(), key=lambda x: x[0]))
print(f"-- -- There are {len(pares_diccionario)} equivalences")
print("-- -- Eq dict constructed in :", time.time() - start)

def replace_keywords(lst, keyword_dict):
    return " ".join([keyword_dict.get(word, word) for word in lst])

start = time.time()
processed["lemmas_split"] = processed['lemmas'].apply(lambda x: x.split())
processed['lemmas'] = processed['lemmas_split'].apply(
    lambda x: replace_keywords(x, pares_diccionario))
processed = processed.drop(columns=['lemmas_split'])
print("-- -- Eq substituted in:", time.time() - start)

-- -- There are 111727 stopwords
-- -- Stops filtered in 3.224813461303711
-- -- There are 2560 equivalences
-- -- Eq dict constructed in : 0.006560087203979492
-- -- Eq substituted in: 5.125868320465088


In [28]:
#########################
# Aux function
#########################
def empieza_prefijo(cpv_cadena, prefijo):
        # Verificar si la entrada no es una cadena, y convertirla a cadena si es necesario
        if not isinstance(cpv_cadena, str):
            cpv_cadena = str(cpv_cadena)
        # Dividir la cadena por comas para obtener una lista de números
        numeros = cpv_cadena.split(',')  
        # Verificar si algún número comienza con el prefijo especificado
        return any(numero.strip().startswith(prefijo) for numero in numeros)

#########################
# Get additional metadata
#########################
#Filtering with 2 first digits of  CPV
processed_with_CPV = path_place_CPV[path_place_CPV['CPV'].apply(lambda x: empieza_prefijo(str(x), '45'))]
#processed.reset_index(inplace=True)
df_merged = pd.merge(processed,processed_with_CPV, how='right', on='identifier') 
#Filtro para quitar las filas cuya columna lemmas sea NaN
place_sin_nan_CPV = df_merged[df_merged['lemmas'].notnull() & (df_merged['lemmas'] != '')]
#print(f"-- -- Data merged: {len(processed)} rows.")
#print(f"-- -- Sample: {processed.head()}")

In [93]:
# Paths to data
path_parquets = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/all_processed")
#This parquet contains the complete data with the CPV code
path_place_CPV = path_parquets / "completo_CPV.parquet"
#This parquet contains the complete data after langid spanish language detection
path_place_esp = path_parquets / "df_esp_langid.parquet"
path_manual_stops = "/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/stopwords_sin_duplicados"
path_eq = "/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/eq.txt"
# Read PLACE files. This files are used to merge with the processed data while keeping the information about the origin of the text (minors, insiders, outsiders)
path_place_CPV = pd.read_parquet(path_place_CPV)
path_place_CPV.reset_index(inplace=True)
print(path_place_CPV.columns)

Index(['identifier', 'id_tm', 'title', 'summary', 'lot_name', 'text', 'origin',
       'CPV', 'lang', 'categoria'],
      dtype='object')


In [94]:
dfs_to_train = []
processed = pd.read_parquet(path_place_esp)
# Inicializar un conjunto de stopwords
stopwords = set()
# Lista para registrar los nombres de los archivos procesados
archivos_procesados = []
# Iterar sobre cada archivo en el directorio especificado
for archivo in os.listdir(path_manual_stops):
    if archivo.endswith('.txt'):
        ruta_completa = os.path.join(path_manual_stops, archivo)
        with open(ruta_completa, 'r', encoding='utf-8') as f:
            stopwords.update(f.read().splitlines())
        archivos_procesados.append(archivo)  # Registrar el archivo procesado
# Imprimir los archivos procesados
print("Archivos .txt procesados:")
for archivo in archivos_procesados:
    print(archivo)
def eliminar_stopwords(fila):
    return ' '.join([palabra for palabra in fila.split() if palabra not in stopwords])

start = time.time()
processed['lemmas'] = processed['lemmas'].apply(eliminar_stopwords)   
print(f"-- -- Stops filtered in {time.time() - start}")

# Reemplazar equivalencias
start = time.time()
pares_diccionario = {}
compiled_regexes = {}
with open(path_eq, 'r') as archivo:
    for linea in archivo:
        linea = linea.strip()
        palabras = linea.split(':')
        if len(palabras) < 2:
            print(f"Línea omitida o incompleta: '{linea}'")
            continue
        pares_diccionario[palabras[0]] = palabras[1]
pares_diccionario = \
    dict(sorted(pares_diccionario.items(), key=lambda x: x[0]))
print("-- -- Eq dict constructed in :", time.time() - start)

def replace_keywords(lst, keyword_dict):
    return " ".join([keyword_dict.get(word, word) for word in lst])

start = time.time()
processed["lemmas_split"] = processed['lemmas'].apply(lambda x: x.split())
processed['lemmas'] = processed['lemmas_split'].apply(
    lambda x: replace_keywords(x, pares_diccionario))
processed = processed.drop(columns=['lemmas_split'])
print("-- -- Eq substituted in:", time.time() - start)

def empieza_prefijo(cpv_cadena, prefijo):
    # Verificar si la entrada no es una cadena, y convertirla a cadena si es necesario
    if not isinstance(cpv_cadena, str):
        cpv_cadena = str(cpv_cadena)
    # Dividir la cadena por comas para obtener una lista de números
    numeros = cpv_cadena.split(',')  
    # Verificar si algún número comienza con el prefijo especificado
    return any(numero.strip().startswith(prefijo) for numero in numeros)

Archivos .txt procesados:
apellidos.txt
companies_stops.txt
tf_idf.txt
nombres.txt
gentilicios.txt
common_stopwords.txt
administracion.txt
ccaa_ciudades.txt
municipios.txt
more_stops.txt
all_more_stops.txt
ministerios.txt
-- -- Stops filtered in 3.6856231689453125
-- -- Eq dict constructed in : 0.004805564880371094
-- -- Eq substituted in: 5.081570148468018


In [97]:
#Filtering with 2 first digits of  CPV
processed_with_CPV = path_place_CPV[path_place_CPV['CPV'].apply(lambda x: empieza_prefijo(str(x), '45'))]
print(processed_with_CPV)
#processed.reset_index(inplace=True)
print("EL processed es:\n",processed)

                                                identifier    id_tm  \
13       contratosMenoresPerfilesContratantes_2018.zip/...       92   
16       contratosMenoresPerfilesContratantes_2018.zip/...      106   
18       contratosMenoresPerfilesContratantes_2018.zip/...      108   
25       contratosMenoresPerfilesContratantes_2018.zip/...      145   
26       contratosMenoresPerfilesContratantes_2018.zip/...      146   
...                                                    ...      ...   
1570177  PlataformasAgregadasSinMenores_202401.zip/Plat...  3360618   
1570189  PlataformasAgregadasSinMenores_202401.zip/Plat...  3360633   
1570191  PlataformasAgregadasSinMenores_202401.zip/Plat...  3360635   
1570205  PlataformasAgregadasSinMenores_202401.zip/Plat...  3360649   
1570210  PlataformasAgregadasSinMenores_202401.zip/Plat...  3360672   

                                                     title  \
13       Pequeñas actuaciones preventivas para evitar a...   
16       Reparación de 

In [ ]:
############################
# Filter by lemmas min len #
############################
min_lemmas = 2
processed['len'] = processed['lemmas'].apply(lambda x: len(x.split()))
processed = processed_with_CPV[processed_with_CPV['len'] > min_lemmas]

df_merged = pd.merge(processed,processed_with_CPV, how='right', on='identifier') 
#Filtro para quitar las filas cuya columna lemmas esté vacía
place_sin_nan_CPV = df_merged[df_merged['lemmas'].notnull() & (df_merged['lemmas'] != '')]
print(len(place_sin_nan_CPV))

In [99]:
#Filtering with 2 first digits of  CPV
processed_with_CPV = path_place_CPV[path_place_CPV['CPV'].apply(lambda x: empieza_prefijo(str(x), '45'))]
#processed.reset_index(inplace=True)
print("EL processed es:\n",processed)
############################
# Filter by lemmas min len #
############################
min_lemmas = 2
processed['len'] = processed['lemmas'].apply(lambda x: len(x.split()))
processed = processed[processed['len'] > min_lemmas]
print("EL processed es:\n",processed)

EL processed es:
                                                 identifier    id_tm  \
0        contratosMenoresPerfilesContratantes_2018.zip/...        0   
1        contratosMenoresPerfilesContratantes_2018.zip/...        1   
2        contratosMenoresPerfilesContratantes_2018.zip/...        2   
3        contratosMenoresPerfilesContratantes_2018.zip/...        4   
4        contratosMenoresPerfilesContratantes_2018.zip/...        5   
...                                                    ...      ...   
2618579  PlataformasAgregadasSinMenores_202401.zip/Plat...  3110256   
2618580  PlataformasAgregadasSinMenores_202401.zip/Plat...  3110257   
2618581  PlataformasAgregadasSinMenores_202401.zip/Plat...  3110258   
2618582  PlataformasAgregadasSinMenores_202401.zip/Plat...  3110259   
2618583  PlataformasAgregadasSinMenores_202401.zip/Plat...  3110260   

                                                  raw_text  \
0        Reforma de elementos de ventilación exterior d...   
1    

In [100]:
df_merged = pd.merge(processed,processed_with_CPV, how='right', on='identifier') 
#Filtro para quitar las filas cuya columna lemmas esté vacía
place_sin_nan_CPV = df_merged[df_merged['lemmas'].notnull() & (df_merged['lemmas'] != '')]
print(len(place_sin_nan_CPV))

49837


In [101]:
place_sin_nan_CPV

,identifier,id_tm_x,raw_text,lemmas,lang_x,len,id_tm_y,title,summary,lot_name,text,origin,CPV,lang_y,categoria
9,contratosMenoresPerfilesContratantes_2018.zip/...,6275.0,Contrato Menor de Obras para la Instalación de...,rejillas foso_salto longitud pista_atletismo e...,es,5.0,297,Contrato Menor de Obras para la Instalación de...,Id licitación: CON-43/17; Órgano de Contrataci...,nan,Contrato Menor de Obras para la Instalación de...,minors,45210000.0,es,Trabajos de construcción
18,contratosMenoresPerfilesContratantes_2018.zip/...,6410.0,Acabado en revestimiento de granito en torre d...,acabado revestimiento granito garaje génesis,es,5.0,720,Acabado en revestimiento de granito en torre d...,Id licitación: GEN-01/2017; Órgano de Contrata...,nan,Acabado en revestimiento de granito en torre d...,minors,"45400000, 45000000",es,Trabajos de construcción & Trabajos de constru...
25,contratosMenoresPerfilesContratantes_2018.zip/...,1204.0,Obras de remodelación de sala y almacen en pla...,almacén inferior cerámica concejalía insercion...,es,7.0,1428,Obras de remodelación de sala y almacen en pla...,Id licitación: 000100/2017-1069; Órgano de Con...,nan,Obras de remodelación de sala y almacen en pla...,minors,45000000.0,es,Trabajos de construcción
32,contratosMenoresPerfilesContratantes_2018.zip/...,1485.0,Realización de trabajos de refuerzo estructura...,estructura tabiquería_pladur anclaje,es,3.0,1690,Realización de trabajos de refuerzo estructura...,Id licitación: 201700000151; Órgano de Contrat...,nan,Realización de trabajos de refuerzo estructura...,minors,45432210.0,es,Trabajos de construcción
35,contratosMenoresPerfilesContratantes_2018.zip/...,785.0,Obras de acondicionamiento de parcela en La Ba...,huerto incorporar_medida relativo_inserción du...,es,4.0,1840,Obras de acondicionamiento de parcela en La Ba...,Id licitación: 000085/2017-1069; Órgano de Con...,nan,Obras de acondicionamiento de parcela en La Ba...,minors,45000000.0,es,Trabajos de construcción
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229058,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109396.0,Obras de la línea eléctrica de alta tensión pa...,alto_tensión etapa ampliado,es,3.0,3360453,Obras de la línea eléctrica de alta tensión pa...,Id licitación: 148-2023;Órgano de Contratación...,nan,Obras de la línea eléctrica de alta tensión pa...,outsiders,"45315400, 45231400",es,Trabajos de construcción & Trabajos de constru...
229061,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109394.0,Obras restaurac puesta en valor cromlech casti...,restaurac cromlech cumbres_mayores,es,3.0,3360477,Obras restaurac puesta en valor cromlech casti...,Id licitación: CONTR 2023 0001016450; Órgano d...,nan,Obras restaurac puesta en valor cromlech casti...,outsiders,45212350.0,es,Trabajos de construcción
229074,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109095.0,Contratación de la ejecución de las obras par...,federaciones euskadi bizkaia bizkaia,es,4.0,3360557,Contratación de la ejecución de las obras par...,Id licitación: k-12/2023; Órgano de Contratac...,nan,Contratación de la ejecución de las obras par...,outsiders,45000000.0,es,Trabajos de construcción
229083,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109376.0,Obras de reforma del parking de Lakua para mej...,lakua mercado_sedentario europea_nextgenerationeu,es,3.0,3360604,Obras de reforma del parking de Lakua para mej...,Id licitación: 2023/EU_SOBR/0062; Órgano de C...,nan,Obras de reforma del parking de Lakua para mej...,outsiders,45220000.0,es,Trabajos de construcción


In [68]:
############################
# Filter by lemmas min len #
############################
min_lemmas = 1
min_lemmas_tm = 2
place_sin_nan_CPV['len'] = place_sin_nan_CPV['lemmas'].apply(lambda x: len(x.split()))
processed = place_sin_nan_CPV[place_sin_nan_CPV['len'] > min_lemmas]

/tmp/ipykernel_3130674/910596984.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  place_sin_nan_CPV['len'] = place_sin_nan_CPV['lemmas'].apply(lambda x: len(x.split()))


In [69]:
processed

,identifier,id_tm_x,raw_text,lemmas,lang_x,id_tm_y,title,summary,lot_name,text,origin,CPV,lang_y,categoria,len
1,contratosMenoresPerfilesContratantes_2018.zip/...,159041.0,Reparación de urgencia en las filtraciones de ...,filtración parque_bomberos,es,106,Reparación de urgencia en las filtraciones de ...,Id licitación: 2018/001699; Órgano de Contrata...,nan,Reparación de urgencia en las filtraciones de ...,minors,45260000.0,es,Trabajos de construcción,2
3,contratosMenoresPerfilesContratantes_2018.zip/...,158973.0,Construcción Depósito para almacenamiento de A...,larga_dotación cuartel,es,145,Construcción Depósito para almacenamiento de A...,Id licitación: P/0769/M/18/6; Órgano de Contra...,nan,Construcción Depósito para almacenamiento de A...,minors,"45300000, 45000000, 45200000",es,Trabajos de construcción & Trabajos de constru...,2
9,contratosMenoresPerfilesContratantes_2018.zip/...,6275.0,Contrato Menor de Obras para la Instalación de...,rejillas foso_salto longitud pista_atletismo e...,es,297,Contrato Menor de Obras para la Instalación de...,Id licitación: CON-43/17; Órgano de Contrataci...,nan,Contrato Menor de Obras para la Instalación de...,minors,45210000.0,es,Trabajos de construcción,5
13,contratosMenoresPerfilesContratantes_2018.zip/...,6392.0,Sustitución de un tramo de red de abastacemien...,abastacemiento puerto,es,432,Sustitución de un tramo de red de abastacemien...,Id licitación: CM53/17OB; Órgano de Contrataci...,nan,Sustitución de un tramo de red de abastacemien...,minors,"71000000, 45000000, 71500000",es,"Servicios de arquitectura, construcción, ingen...",2
18,contratosMenoresPerfilesContratantes_2018.zip/...,6410.0,Acabado en revestimiento de granito en torre d...,acabado revestimiento granito garaje génesis,es,720,Acabado en revestimiento de granito en torre d...,Id licitación: GEN-01/2017; Órgano de Contrata...,nan,Acabado en revestimiento de granito en torre d...,minors,"45400000, 45000000",es,Trabajos de construcción & Trabajos de constru...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229078,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109086.0,Sustitución de equipos generadores en sala de ...,sala_caldera receptor,es,3360567,Sustitución de equipos generadores en sala de ...,Id licitación: 042/2024;Órgano de Contratación...,nan,Sustitución de equipos generadores en sala de ...,outsiders,45331000.0,es,Trabajos de construcción,2
229080,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109378.0,Implantación de arbolado en el barrio de Goiko...,arbolado goikolarra,es,3360601,Implantación de arbolado en el barrio de Goiko...,Id licitación: 2023/CO_SOBR/0060; Órgano de C...,nan,Implantación de arbolado en el barrio de Goiko...,outsiders,45112710.0,es,Trabajos de construcción,2
229083,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109376.0,Obras de reforma del parking de Lakua para mej...,lakua mercado_sedentario europea_nextgenerationeu,es,3360604,Obras de reforma del parking de Lakua para mej...,Id licitación: 2023/EU_SOBR/0062; Órgano de C...,nan,Obras de reforma del parking de Lakua para mej...,outsiders,45220000.0,es,Trabajos de construcción,3
229084,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109375.0,Obras de infraestructura verde en el casco med...,medieval ttipia,es,3360605,Obras de infraestructura verde en el casco med...,Id licitación: 2023/EU_SOBR/0061; Órgano de C...,nan,Obras de infraestructura verde en el casco med...,outsiders,45220000.0,es,Trabajos de construcción,2


In [70]:
all = processed.copy()
#minors = all[all.origen == "minors"]
#outsiders = all[all.origen == "outsiders"]
#insiders = all[all.origen == "insiders"]

In [71]:
# Get tenders that were not included in the modeling process
all_not_tm = all[all['len'] <= min_lemmas_tm]
all_not_tm

,identifier,id_tm_x,raw_text,lemmas,lang_x,id_tm_y,title,summary,lot_name,text,origin,CPV,lang_y,categoria,len
1,contratosMenoresPerfilesContratantes_2018.zip/...,159041.0,Reparación de urgencia en las filtraciones de ...,filtración parque_bomberos,es,106,Reparación de urgencia en las filtraciones de ...,Id licitación: 2018/001699; Órgano de Contrata...,nan,Reparación de urgencia en las filtraciones de ...,minors,45260000.0,es,Trabajos de construcción,2
3,contratosMenoresPerfilesContratantes_2018.zip/...,158973.0,Construcción Depósito para almacenamiento de A...,larga_dotación cuartel,es,145,Construcción Depósito para almacenamiento de A...,Id licitación: P/0769/M/18/6; Órgano de Contra...,nan,Construcción Depósito para almacenamiento de A...,minors,"45300000, 45000000, 45200000",es,Trabajos de construcción & Trabajos de constru...,2
13,contratosMenoresPerfilesContratantes_2018.zip/...,6392.0,Sustitución de un tramo de red de abastacemien...,abastacemiento puerto,es,432,Sustitución de un tramo de red de abastacemien...,Id licitación: CM53/17OB; Órgano de Contrataci...,nan,Sustitución de un tramo de red de abastacemien...,minors,"71000000, 45000000, 71500000",es,"Servicios de arquitectura, construcción, ingen...",2
19,contratosMenoresPerfilesContratantes_2018.zip/...,5920.0,"Contrato Menor de Servicios para la ""Reforma d...",estadio_juegos mediterráneos,es,744,"Contrato Menor de Servicios para la ""Reforma d...",Id licitación: CON-15/17; Órgano de Contrataci...,nan,"Contrato Menor de Servicios para la ""Reforma d...",minors,45200000.0,es,Trabajos de construcción,2
20,contratosMenoresPerfilesContratantes_2018.zip/...,5516.0,Obra de sustitución de puertas de acceso desde...,nuevo plataforma_elevación,es,1085,Obra de sustitución de puertas de acceso desde...,Id licitación: 201700000445; Órgano de Contrat...,nan,Obra de sustitución de puertas de acceso desde...,minors,45262520.0,es,Trabajos de construcción,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229051,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109210.0,Abastecimiento a Zandio y Osacáin (AM273273_01),zandio osacáin,es,3360429,Abastecimiento a Zandio y Osacáin (AM273273_01),Id licitación: 3167_2023/SINF-PRO/000012; Órga...,nan,Abastecimiento a Zandio y Osacáin (AM273273_01...,outsiders,"45231300, 45213260",es,Trabajos de construcción & Trabajos de constru...,2
229063,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109163.0,Ejecución de las obras correspondientes a la F...,urbanización_subámbito iurramendi,es,3360484,Ejecución de las obras correspondientes a la F...,Id licitación: F2-3; Órgano de Contratación: ...,nan,Ejecución de las obras correspondientes a la F...,outsiders,45210000.0,es,Trabajos de construcción,2
229078,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109086.0,Sustitución de equipos generadores en sala de ...,sala_caldera receptor,es,3360567,Sustitución de equipos generadores en sala de ...,Id licitación: 042/2024;Órgano de Contratación...,nan,Sustitución de equipos generadores en sala de ...,outsiders,45331000.0,es,Trabajos de construcción,2
229080,PlataformasAgregadasSinMenores_202401.zip/Plat...,3109378.0,Implantación de arbolado en el barrio de Goiko...,arbolado goikolarra,es,3360601,Implantación de arbolado en el barrio de Goiko...,Id licitación: 2023/CO_SOBR/0060; Órgano de C...,nan,Implantación de arbolado en el barrio de Goiko...,outsiders,45112710.0,es,Trabajos de construcción,2


In [56]:
#all[all['len'] > min_lemmas_tm]

,identifier,id_tm,raw_text,lemmas,lang,len
25,contratosMenoresPerfilesContratantes_2018.zip/...,27,Obras de sustitución de sistemas de producción...,subdelegación_gobierno cuerpo_guardia secretar...,es,7
32,contratosMenoresPerfilesContratantes_2018.zip/...,34,Diseño de la escenografía de El burlador de Se...,diseño_escenografía burlador kermesse heroica ...,es,3
37,contratosMenoresPerfilesContratantes_2018.zip/...,40,Suministro de mobiliario para dependencias del...,organismo comisaria_aguas oficinas_centrales e...,es,5
38,contratosMenoresPerfilesContratantes_2018.zip/...,41,Reparación de estaciones de aforo permanentes ...,permanente carretera,es,4
41,contratosMenoresPerfilesContratantes_2018.zip/...,44,Retirada de cercados de alambre en el entorno ...,retirado alambre embalse_ebro,es,3
...,...,...,...,...,...,...
229009,contratosMenoresPerfilesContratantes_2019.zip/...,253676,Servicio de producción y difusión publicidad i...,consejería_fomento radiofónico,es,3
229044,contratosMenoresPerfilesContratantes_2019.zip/...,253722,"Maquetación, impresión, encuadernación y trans...",maquetación_impresión encuadernación prueba_se...,es,4
229061,contratosMenoresPerfilesContratantes_2019.zip/...,253740,Servicios de impresión de la publicación Touri...,tourist_info news,es,3
229083,contratosMenoresPerfilesContratantes_2019.zip/...,253768,Parque infantil 21 de enero compuesto por hinc...,compuesto_hinchabl juego_tradicional,es,3


## CPV


In [72]:
path_model = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_sin_nan_CPV_15_topics_45_FINAL/")
docs = all_not_tm[["identifier", "lemmas"]].values
num_topics = 15

In [73]:
print(docs)

[['contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/301'
  'filtración parque_bomberos']
 ['contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes.atom/370'
  'larga_dotación cuartel']
 ['contratosMenoresPerfilesContratantes_2018.zip/contratosMenoresPerfilesContratantes_20190225_140722.atom/27'
  'abastacemiento puerto']
 ...
 ['PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores_20240201_040018_1.atom/369'
  'sala_caldera receptor']
 ['PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores_20240201_040018_1.atom/49'
  'arbolado goikolarra']
 ['PlataformasAgregadasSinMenores_202401.zip/PlataformasAgregadasSinMenores_20240201_040018_1.atom/52'
  'medieval ttipia']]


In [74]:
len(all_not_tm)

47074

In [79]:
infer_thetas(path_model, num_topics, docs)

-- -- Mallet corpus.txt for inference created.
-- Extracting pipeline
-- Running command /export/usuarios_ml4ds/cggamella/NP-Search-Tool/Mallet-202108/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_sin_nan_CPV_15_topics_45_FINAL/train_data/corpus.mallet --input /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_sin_nan_CPV_15_topics_45_FINAL/train_data/corpus_aux.txt --output /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_sin_nan_CPV_15_topics_45_FINAL/train_data/import.pipe
-- Inference: Mallet Data Import
-- Running command /export/usuarios_ml4ds/cggamella/NP-Search-Tool/Mallet-202108/bin/mallet import-file --use-pipe-from /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_sin_nan_CPV_15_topics_45_FINAL/train_data/import.pipe --input /export/usuarios_ml4ds/cggamella/NP-Search-Tool/sam

(47074, 15)